<a href="https://colab.research.google.com/github/Jarvis1000x/Facial_Recognition/blob/main/Face_Verification_with_Siamese_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Standard Dependencies
import os
import cv2
import random
import numpy as np
from matplotlib import pyplot as plt
import uuid

# Tensorflow Dependencies
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [2]:
# Avoiding OOM Errors for GPU
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Loading and Preprocessing Data

In [3]:
# Folder Structure
Pos_Path = "/content/drive/MyDrive/Colab Notebooks/Facial Verification/data/positive"
Neg_Path = "/content/drive/MyDrive/Colab Notebooks/Facial Verification/data/negative"
Anc_Path = "/content/drive/MyDrive/Colab Notebooks/Facial Verification/data/anchor"

In [4]:
def load_dataset(filepath):
    print(filepath+'/*.jpg')
    train_list_ds = tf.data.Dataset.list_files(filepath+'/*.jpg', shuffle=False).take(300)
    return train_list_ds

anchor = load_dataset(Anc_Path)
positive = load_dataset(Pos_Path)
negative = load_dataset(Neg_Path)

/content/drive/MyDrive/Colab Notebooks/Facial Verification/data/anchor/*.jpg
/content/drive/MyDrive/Colab Notebooks/Facial Verification/data/positive/*.jpg
/content/drive/MyDrive/Colab Notebooks/Facial Verification/data/negative/*.jpg


In [5]:
# Preprocessing
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (100, 100))
    img = img /255.0
    return img    

In [6]:
# Creating Labelled Dataset
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [7]:
def preprocess_twins(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [8]:
# Dataloader Pipeline
data = data.map(preprocess_twins)
data = data.cache()
data = data.shuffle(buffer_size=1024)

# Training Partition 
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [9]:
# Testing Partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

# Building the Model

In [10]:
# Embedding Layer
def make_embedding():
    inp = Input(shape=(100, 100, 3), name="input_image")
    
    # First Block
    c1 = Conv2D(64, (10, 10), activation="relu")(inp)
    m1 = MaxPooling2D(64, (2, 2), padding="same")(c1)
    
    #Second Block
    c2 = Conv2D(128, (7, 7), activation="relu")(m1)
    m2 = MaxPooling2D(64, (2, 2), padding="same")(c2)
    
    #Third Block
    c3 = Conv2D(128, (4, 4), activation="relu")(m2)
    m3 = MaxPooling2D(64, (2, 2), padding="same")(c2)
    
    # Final Embedding Layer
    c4 = Conv2D(256, (4, 4), activation="relu")(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation="sigmoid")(f1)
    
    return Model(inputs=[inp], outputs=[d1], name="embedding")

In [11]:
embedding = make_embedding()

In [12]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     [(None, 100, 100, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 91, 91, 64)        19264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 128)       401536    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 17, 17, 256)       524544    
_________________________________________________________________
flatten (Flatten)            (None, 73984)             0 

In [13]:
# Siamese L1 Distance Class
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    
    # Similarity Clarification
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [14]:
l1 = L1Dist()

In [15]:
# Siamese Model
def make_siamese_model():
    
    # Anchor Image
    input_image = Input(name="input_img", shape=(100, 100, 3))
    
    # Validation Image
    validation_image = Input(name="validation_img", shape=(100, 100, 3))
    
    # Siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = "distance"
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification Layer
    classifier = Dense(1, activation="sigmoid")(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name="SiameseNetwork")


In [16]:
siamese_model = make_siamese_model()

In [17]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
validation_img (InputLayer)     [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
embedding (Functional)          (None, 4096)         303987904   input_img[0][0]                  
                                                                 validation_img[0][0]             
__________________________________________________________________________________________________
distance (L1Dist)               (None, 4096)         0           embedding[0][0]     

# Training

In [18]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [19]:
opt = tf.keras.optimizers.Adam(1e-4)

In [20]:
# Checkpoints
checkpoint_dir = '/content/drive/MyDrive/Colab Notebooks/Facial Verification/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [25]:
@tf.function
def train_step(batch):
    
    with tf.GradientTape() as tape:
        # Anchor & Positive/Negative image
        X = batch[:2]
        
        # Label
        Y = batch[2]
        
        # Forward Pass
        yhat = siamese_model(X, training=True)
        # Calculate loss
        loss = binary_cross_loss(Y, yhat)
    print(loss)
        
    # Calculate Gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    
    return loss

In [22]:
# Training Loop
def train(data, epochs):
    for epoch in range(1, epochs+1):
        print("\n Epoch {}/{}".format(epoch, epochs))
        progbar = tf.keras.utils.Progbar(len(data))
        
        for idx, batch in enumerate(data):
            train_step(batch)
            progbar.update(idx+1)
            
        # Save Checkpoints
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

In [23]:
EPOCHS = 50

In [26]:
train(train_data, EPOCHS)


 Epoch 1/50
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
27/27 [==============================] - 56s 899ms/step

 Epoch 2/50
27/27 [==============================] - 23s 901ms/step

 Epoch 3/50
27/27 [==============================] - 23s 889ms/step

 Epoch 4/50
27/27 [==============================] - 23s 881ms/step

 Epoch 5/50
27/27 [==============================] - 23s 880ms/step

 Epoch 6/50
27/27 [==============================] - 23s 876ms/step

 Epoch 7/50
27/27 [==============================] - 23s 876ms/step

 Epoch 8/50
27/27 [==============================] - 23s 871ms/step

 Epoch 9/50
27/27 [==============================] - 23s 870ms/step

 Epoch 10/50
27/27 [==============================] - 23s 874ms/step

 Epoch 11/50
27/27 [==============================] - 23s 883ms/step

 Epoch 12/50
27/27 [==============================] - 23s 886ms/step

 Epoch 13/50
2

# Evaluating Model

In [27]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [28]:
# Batch of Test Data
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [30]:
# Making Predictions
y_hat = siamese_model.predict([test_input, test_val])

# Post Processing the results
[1 if prediction > 0.5 else 0 for prediction in y_hat]

[1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1]

In [31]:
y_true

array([1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1.],
      dtype=float32)

In [32]:
# Metric Object
m = Recall()

# Calculating Recall Value
m.update_state(y_true, y_hat)

# Return Recall Results
m.result().numpy()

1.0

In [33]:
# Metric Object
m = Precision()

# Calculating Recall Value
m.update_state(y_true, y_hat)

# Return Recall Results
m.result().numpy()

1.0

In [ ]:
# Visualize Results
plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
plt.imshow(test_input[2])
plt.subplot(1, 2, 2)
plt.imshow(test_val[2])
plt.show

# Saving the Model

In [37]:
# Save Weights
siamese_model.save("siamesemodel.h5")

In [38]:
# Reload the Model
model = tf.keras.models.load_model("siamesemodel.h5",
                                   custom_objects={"L1Dist":L1Dist, "BinaryCrossentropy":tf.losses.BinaryCrossentropy})